In [1]:
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
from statsmodels.api import OLS
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import chain
import seaborn 
import pickle
import sklearn.metrics as sk
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor

## Pattern Statistics (all and experiment only) ##

In [28]:
## Extract Pattern Data ##
patterns_file = open(os.getcwd()+"/data/patterns.pkl", 'rb')
patterns = pickle.load(patterns_file)
patterns_file.close()
print(len(patterns))

1378


In [9]:
## Extract experiment data ##
data = []
tap_file = os.getcwd()+"/data/tap_rhythm_contour_raw.csv"
with open(tap_file) as results: 
    reader = csv.reader(results)
    for row in reader:
        data.append(row)
    results.close()


## Experiment Statistics ##

In [27]:
## Clean csv file (if necessary) ## 
tap_data = []
for row in data:
    line = row[2:20]
    tap_data.append(line)
for i in range(3):
    print(tap_data[i])

['ID', 'NAME', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16']
['1355', 'breakbeat_unconventional_DMR_1', '0.716535433', '0', '0', '0', '0.661417323', '0', '0.700787402', '0', '0', '0', '0.716535433', '0', '0.732283465', '0', '0', '0']
['1043', 'chicago_promisedland_138', '0.496062992', '0', '0.417322835', '0', '0.488188976', '0', '0', '0', '0.42519685', '0', '0.456692913', '0', '0.480314961', '0', '0', '0']
